# Notebook de Scraping Complet des Dépôts GitHub

**Objectif :** Ce notebook a pour unique but de scraper des informations détaillées sur des dépôts GitHub à partir d'un fichier JSON. Il est conçu pour des extractions longues (plusieurs heures) et inclut :
- La récupération des métadonnées de base.
- L'extraction du contenu complet du fichier **README.md** de chaque dépôt.
- Une sauvegarde progressive dans un fichier CSV pour plus de robustesse.

In [ ]:
import requests
import pandas as pd
import json
import ijson
import os
import gc
import base64
from tqdm.notebook import tqdm
from time import sleep

# --- CONFIGURATION ---

# ⚠️ ATTENTION : Ne jamais stocker de token en clair dans un notebook partagé.
# Pour une utilisation personnelle, remplacez la valeur ci-dessous.
TOKEN = os.getenv("GITHUB_TOKEN")
HEADERS = {'Authorization': f'token {TOKEN}', 'Accept': 'application/vnd.github.v3+json'}

# Fichiers
INPUT_FILE = "repos-min-1000stars.json"
OUTPUT_FILE = "github_data_with_readmes.csv" # Nom du fichier de sortie final

# Paramètres du script
BATCH_SIZE = 500      # Sauvegarde tous les N dépôts. Crucial pour les longs scripts.
API_SLEEP = 0.5       # Temps d'attente en secondes entre chaque appel API pour éviter le rate limiting.

In [11]:
def json_stream(file_path):
    """Lit un gros fichier JSON de manière itérative, sans limite."""
    with open(file_path, "r", encoding="utf-8") as f:
        # ijson.items est un générateur, parfait pour les gros fichiers
        for repo in ijson.items(f, "item"):
            yield repo

def get_api_json(url):
    """Fait une requête GET à l'API GitHub avec gestion du rate limiting."""
    try:
        r = requests.get(url, headers=HEADERS)
        if r.status_code == 200:
            return r.json()
        elif r.status_code == 403:
            print("⏳ Rate limit de l'API atteint, pause de 60 secondes...")
            sleep(60)
            return get_api_json(url) # On réessaie la même requête
        elif r.status_code == 404:
            # C'est une erreur normale si un README n'existe pas, on ne l'affiche pas pour ne pas polluer la sortie.
            return None
        else:
            print(f"⚠️ Code d'erreur {r.status_code} pour l'URL {url}")
            return None
    except Exception as e:
        print(f"💥 Erreur de connexion sur {url}: {e}")
        return None

def count_json_items(file_path):
    """Compte rapidement le nombre d'éléments à la racine d'un fichier JSON."""
    print("Pré-calcul du nombre total de dépôts (cela peut prendre un moment)...")
    with open(file_path, 'rb') as f:
        # On itère rapidement sur le fichier juste pour compter
        total = sum(1 for _ in ijson.items(f, 'item'))
    print(f"Total trouvé : {total} dépôts.")
    return total
    
def get_readme_content(repo_full_name):
    """Récupère et décode le contenu du README pour un dépôt donné."""
    readme_url = f"https://api.github.com/repos/{repo_full_name}/readme"
    readme_data = get_api_json(readme_url)
    
    if readme_data and 'content' in readme_data:
        # Le contenu est encodé en Base64 par l'API GitHub
        content_base64 = readme_data['content']
        try:
            # On décode le Base64 en bytes, puis les bytes en string utf-8
            decoded_bytes = base64.b64decode(content_base64)
            return decoded_bytes.decode('utf-8')
        except Exception as e:
            print(f"Impossible de décoder le README pour {repo_full_name}: {e}")
            return "" # On retourne une chaîne vide en cas d'erreur de décodage
    return "" # Pas de README trouvé ou pas de champ 'content'

print("✅ Fonctions prêtes.")

✅ Fonctions prêtes.


In [ ]:
# Liste pour stocker les données d'un lot (batch)
total_repos = count_json_items(INPUT_FILE)
data_to_save = []

# Si le fichier de sortie existe déjà d'une précédente exécution, on le supprime pour repartir de zéro.
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)
    print(f"Fichier de sortie existant '{OUTPUT_FILE}' supprimé.")

# On utilise un générateur, donc on ne connaît pas la longueur totale à l'avance.
# tqdm affichera le nombre d'itérations.
repo_generator = json_stream(INPUT_FILE)

# On ajoute un compteur manuel pour la sauvegarde par lots
for i, repo in enumerate(tqdm(repo_generator, total=total_repos, desc="Scraping des dépôts GitHub")):

    # 1. Collecte des données de base depuis le JSON
    repo_data = {
        "full_name": repo.get("full_name"),
        "language": repo.get("language"),
        "stars": repo.get("stargazers_count"),
        "forks": repo.get("forks_count"),
        "description": repo.get("description"),
        "created_at": repo.get("created_at"),
        "html_url": repo.get("html_url"),
    }
    
    # 2. Récupération du contenu du README (1 appel API par dépôt)
    if repo.get("full_name"):
        repo_data["readme_content"] = get_readme_content(repo["full_name"])
        sleep(API_SLEEP) # Pause pour respecter l'API
    else:
        repo_data["readme_content"] = ""

    data_to_save.append(repo_data)
    
    # 3. Sauvegarde par lot pour la robustesse
    if (i + 1) % BATCH_SIZE == 0:
        df_batch = pd.DataFrame(data_to_save)
        
        # On écrit le header seulement si le fichier n'existe pas encore
        write_header = not os.path.exists(OUTPUT_FILE)
        
        # On ajoute les données au CSV (mode='a' pour append)
        df_batch.to_csv(OUTPUT_FILE, mode='a', index=False, header=write_header, encoding='utf-8-sig')
        
        print(f"\n💾 Lot de {len(data_to_save)} dépôts sauvegardé dans '{OUTPUT_FILE}'. Total traité : {i + 1}")
        
        # On vide la liste et on libère la mémoire
        data_to_save.clear()
        gc.collect()

# --- Sauvegarde Finale ---
# N'oubliez pas de sauvegarder le dernier lot qui n'atteint peut-être pas la taille du BATCH_SIZE
if data_to_save:
    print(f"\n💾 Sauvegarde du dernier lot de {len(data_to_save)} dépôts...")
    df_final_batch = pd.DataFrame(data_to_save)
    write_header = not os.path.exists(OUTPUT_FILE)
    df_final_batch.to_csv(OUTPUT_FILE, mode='a', index=False, header=write_header, encoding='utf-8-sig')
    data_to_save.clear()
    gc.collect()

print(f"\n\n🎉 --- SCRAPING TERMINÉ --- 🎉")
print(f"Toutes les données ont été sauvegardées dans le fichier '{OUTPUT_FILE}'.")

Pré-calcul du nombre total de dépôts (cela peut prendre un moment)...
Total trouvé : 56861 dépôts.


Scraping des dépôts GitHub:   0%|          | 0/56861 [00:00<?, ?it/s]


💾 Lot de 500 dépôts sauvegardé dans 'github_data_with_readmes.csv'. Total traité : 500


In [14]:
# Une fois le scraping terminé, vous pouvez exécuter cette cellule pour vérifier le résultat

try:
    df_final = pd.read_csv(OUTPUT_FILE)
    print(f"Le fichier '{OUTPUT_FILE}' a été chargé avec succès.")
    print(f"Nombre total de dépôts scrapés : {len(df_final)}")
    print("Aperçu des données :")
    
    # On affiche l'aperçu sans le contenu du README pour plus de lisibilité
    display(df_final.drop(columns=['readme_content']).head())
    
    # On vérifie qu'un README a bien été récupéré
    print("\nExemple de début de README pour le premier dépôt :")
    # Affiche les 300 premiers caractères du premier README non vide
    first_readme = df_final[df_final['readme_content'].notna()]['readme_content'].iloc[0]
    print(first_readme[:300] + "...")

except FileNotFoundError:
    print(f"Le fichier de sortie '{OUTPUT_FILE}' n'a pas encore été créé. Lancez d'abord le pipeline de scraping.")
except Exception as e:
    print(f"Une erreur est survenue lors de la lecture du fichier CSV : {e}")

Le fichier 'github_data_with_readmes.csv' a été chargé avec succès.
Nombre total de dépôts scrapés : 56861
Aperçu des données :


,full_name,language,stars,forks,description,created_at,html_url
0,mdbootstrap/bootstrap-toggle-buttons,JavaScript,1013,89,Bootstrap-toggle-buttons has moved to https://...,2012-07-17 22:46:54+00:00,https://github.com/mdbootstrap/bootstrap-toggl...
1,cloudfuji/kandan,JavaScript,1005,117,A Cloudfuji chat application,2012-03-06 16:16:28+00:00,https://github.com/cloudfuji/kandan
2,doug/depthjs,C++,1003,106,DepthJS allows any web page to interact with t...,2010-11-19 23:02:54+00:00,https://github.com/doug/depthjs
3,codrops/ModalWindowEffects,JavaScript,1015,232,A set of experimental modal window appearance ...,2013-07-02 08:10:00+00:00,https://github.com/codrops/ModalWindowEffects
4,adactio/Pattern-Primer,CSS,1002,152,Generating styled markup from a folder of mark...,2011-11-18 12:03:32+00:00,https://github.com/adactio/Pattern-Primer



Exemple de début de README pour le premier dépôt :
Bootstrap-toggle-buttons

Bootstrap-toggle-buttons has moved to https://github.com/nostalgiaz/bootstrap-switch

Old doc -> https://github.com/nostalgiaz/bootstrap-toggle-buttons/blob/master/README_OLD.md
...
